In [3]:
import pandas as pd
# from rake_nltk import Rake
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

df = pd.read_excel('Online Retail.xlsx')

In [4]:
from sklearn.model_selection import train_test_split

In [5]:
import seaborn as sns 
import matplotlib.pyplot as plt

In [6]:
df.dropna(inplace=True)

In [7]:
sub = df.sample(n=10000, random_state=1)

In [8]:
sub.head(2)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
443133,574691,22341,LOVE GARLAND PAINTED ZINC,3,2011-11-06 13:17:00,1.65,14174.0,United Kingdom
415129,572517,22562,MONSTERS STENCIL CRAFT,2,2011-10-24 14:47:00,1.25,15068.0,United Kingdom


In [9]:
thing = sub.groupby(['CustomerID', 'StockCode'], as_index=False).count()

In [10]:
grouped_index = df.groupby(['CustomerID', 'StockCode'], as_index=False).size()

In [11]:
grouped_index['StockCode'] = grouped_index['StockCode'].astype(str)

In [12]:
grouped_index['StockCode'] += ' '

In [13]:
final = grouped_index.groupby(['CustomerID'], as_index=False)['StockCode'].sum()

In [14]:
final.head(2)

,CustomerID,StockCode
0,12346.0,23166
1,12347.0,16008 17021 20665 20719 20780 20782 20966 2103...


In [15]:
def tokenize(text):
    tokens = text.split()
    return tokens

In [16]:
final['StockCode'] = final['StockCode'].apply(lambda x: tokenize(x))

In [17]:
final

,CustomerID,StockCode
0,12346.0,[23166]
1,12347.0,"[16008, 17021, 20665, 20719, 20780, 20782, 209..."
2,12348.0,"[21211, 21213, 21725, 21726, 21967, 21977, 219..."
3,12349.0,"[20685, 20914, 21086, 21136, 21231, 21232, 214..."
4,12350.0,"[20615, 20652, 21171, 21832, 21864, 21866, 219..."
...,...,...
4367,18280.0,"[22084, 22180, 22358, 22467, 22495, 22499, 226..."
4368,18281.0,"[22028, 22037, 22467, 22716, 23007, 23008, 23209]"
4369,18282.0,"[21108, 21109, 21270, 22089, 22423, 22424, 226..."
4370,18283.0,"[20676, 20712, 20717, 20718, 20719, 20723, 207..."


Build Word2Vec

In [18]:
def create_unique_word_dict(text:list) -> dict:
    """
    A method that creates a dictionary where the keys are unique words
    and key values are indices
    """
    # Getting all the unique words from our text and sorting them alphabetically
    words = list(set(text))
    words.sort()

    # Creating the dictionary for the unique words
    unique_word_dict = {}
    for i, word in enumerate(words):
        unique_word_dict.update({
            word: i
        })

    return unique_word_dict 

In [ ]:
from scipy import sparse
import numpy as np

# Defining the number of features (unique words)
n_words = len(unique_word_dict)

# Getting all the unique words 
words = list(unique_word_dict.keys())

# Creating the X and Y matrices using one hot encoding
X = []
Y = []

for i, word_list in tqdm(enumerate(word_lists)):
    # Getting the indices
    main_word_index = unique_word_dict.get(word_list[0])
    context_word_index = unique_word_dict.get(word_list[1])

    # Creating the placeholders   
    X_row = np.zeros(n_words)
    Y_row = np.zeros(n_words)

    # One hot encoding the main word
    X_row[main_word_index] = 1

    # One hot encoding the Y matrix words 
    Y_row[context_word_index] = 1

    # Appending to the main matrices
    X.append(X_row)
    Y.append(Y_row)

# Converting the matrices into an array
X = np.asarray(X)
Y = np.asarray(Y)

In [ ]:
# Deep learning: 
from tensorflow.keras.models import Model

from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense

# Defining the size of the embedding
embed_size = 2

# Defining the neural network
inp = Input(shape=(X.shape[1],))
x = Dense(units=embed_size, activation='linear')(inp)
x = Dense(units=Y.shape[1], activation='softmax')(x)
model = Model(inputs=inp, outputs=x)
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam')

# Optimizing the network weights
model.fit(
    x=X, 
    y=Y, 
    batch_size=256,
    epochs=1000
    )

# Obtaining the weights from the neural network. 
# These are the so called word embeddings

# The input layer 
weights = model.get_weights()[0]

# Creating a dictionary to store the embeddings in. The key is a unique word and 
# the value is the numeric vector
embedding_dict = {}
for word in words: 
    embedding_dict.update({
        word: weights[unique_word_dict.get(word)]
        })